In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [2]:
# 测试集：83个音频特征数据读入
# test_path = r".\input\83_sound_features.xlsx"
test_path = r".\input\44_sound_features.xlsx"
train_path = r".\input\6747_sound_features.xls"
test_data = pd.read_excel(test_path)
train_data = pd.read_excel(train_path)
test_data

,Unnamed: 0,25s文件,Leq_mean,Leq_std,Leq_25,Leq_median,Leq_75,Leq_10-Leq_90,Loudness_mean,Loudness_std,...,Tonality_mean,Tonality_std,Tonality_25,Tonality_median,Tonality_75,Tonality_10-Tonality_90,pctn,pctu,pctm,level
0,0,37-L-1.wav,60.87,3.38,58.86,61.67,63.08,-7.71,13.72,2.85,...,0.02,0.03,0.00,0.00,0.04,-0.06,0.800000,0.200000,0.000000,1
1,1,142-L-1.wav,58.14,1.88,56.80,58.17,59.31,-4.58,10.44,2.14,...,0.02,0.02,0.00,0.01,0.03,-0.04,1.000000,0.000000,0.000000,1
2,2,40-L-1.wav,62.21,2.11,60.68,61.68,63.29,-5.54,14.21,2.32,...,0.02,0.03,0.00,0.00,0.04,-0.06,1.000000,0.000000,0.000000,1
3,3,32-L-1.wav,50.14,1.63,49.02,50.01,51.17,-4.10,5.89,0.59,...,0.04,0.03,0.00,0.03,0.06,-0.08,0.571429,0.428571,0.000000,1
4,4,112-L-1.wav,64.17,1.40,63.13,64.23,65.26,-3.55,14.90,1.52,...,0.02,0.03,0.00,0.02,0.04,-0.06,1.000000,0.000000,0.000000,1
5,5,127-L-1.wav,56.94,0.71,56.44,56.93,57.40,-1.83,10.15,0.69,...,0.03,0.02,0.02,0.03,0.05,-0.05,1.000000,0.000000,0.000000,1
6,6,47-L-1.wav,46.18,2.88,43.88,45.45,48.01,-7.17,4.20,0.70,...,0.03,0.04,0.00,0.02,0.05,-0.08,0.833333,0.166667,0.000000,2
7,7,149-L-1.wav,57.77,3.61,54.82,57.81,60.33,-9.34,10.61,3.21,...,0.01,0.02,0.00,0.00,0.01,-0.04,0.900000,0.100000,0.000000,1
8,8,435-L-1.wav,34.52,2.75,32.72,34.28,35.67,-6.09,1.57,0.63,...,0.04,0.05,0.00,0.03,0.06,-0.11,1.000000,0.000000,0.000000,2
9,9,1-L-1.wav,49.88,1.39,48.84,49.75,50.67,-3.56,6.41,0.75,...,0.06,0.05,0.03,0.06,0.09,-0.12,0.615385,0.384615,0.000000,3


In [3]:
test_data.shape


(44, 42)

In [4]:
# 36维特征 + level(暂时还没有）
column_36 = ['Leq_mean', 'Leq_std', 'Leq_25', 'Leq_median',
             'Leq_75', 'Leq_10-Leq_90', 'Loudness_mean', 'Loudness_std',
             'Loudness_25', 'Loudness_median', 'Loudness_75',
             'Loudness_10-Loudness_90', 'Roughness_mean', 'Roughness_std',
             'Roughness_25', 'Roughness_median', 'Roughness_75',
             'Roughness_10-Roughness_90', 'Sharpness_mean', 'Sharpness_std',
             'Sharpness_25', 'Sharpness_median', 'Sharpness_75',
             'Sharpness_10-Sharpness_90', 'Fluct_mean', 'Fluct_std', 'Fluct_25',
             'Fluct_median', 'Fluct_75', 'Fluct_10-Fluct_90', 'Tonality_mean',
             'Tonality_std', 'Tonality_25', 'Tonality_median', 'Tonality_75',
             'Tonality_10-Tonality_90', 'level']
len(column_36)

37

In [5]:
feature_use = column_36
# test_x只取前36个
test_x = test_data[feature_use[:-1]]
test_y = test_data[feature_use[-1]]

# train同理
train_x = train_data[feature_use[:-1]]
train_y = train_data[feature_use[-1]]

In [6]:
# 将数据进行l2正则化
test_x = normalize(test_x, norm="l2")
train_x = normalize(train_x, norm="l2")

In [7]:
len(train_x), len(train_y)

(6747, 6747)

In [8]:
# 分类器重写
def classifier(nb_class):
    """
    分类器集合
    :param nb_class: 类别数
    :return:分类器列表
    """
    clf = [
        KNeighborsClassifier(nb_class),  # 0.68
        DecisionTreeClassifier(max_depth=40),  # 0.69
        SVC(C=10, kernel='rbf', class_weight={1: 8, 2: 4, 3: 1, 4: 10}),  # 0.42
        ExtraTreesClassifier(n_estimators=80, max_depth=24, min_samples_split=3, max_features='auto', random_state=0),
        # 0.825
        # 'max_depth': 21, 'min_samples_split': 2, 'n_estimators': 95}
        RandomForestClassifier(n_estimators=95, max_depth=21, min_samples_split=2, random_state=0),  # 0.758
        # 'max_depth': 11, 'n_estimators': 110
        # XGBClassifier(n_estimators=110, max_depth=11, learning_rate=0.001, objective='multi:softmax', nb_class=4),
        LogisticRegression(penalty='l2', class_weight='balanced', solver='lbfgs')]  # 0.45

    return clf

In [9]:
# 我也没搞懂这里要干啥
nb_class = 4
clf_list = classifier(nb_class)

# 0: KNN    1: 决策树    2: SVC
# 3: 极端树  4: 随机森林  5: LogisticRegression
clf = clf_list[0]
# 训练分类器
clf.fit(train_x, train_y)

KNeighborsClassifier(n_neighbors=4)

In [10]:
# 导入模型
joblib.dump(clf, r'./model/model_0516.pkl')
pre = clf.predict(test_x)

In [11]:
# 计算正确率
# 有+-1
# def accuracy(test_y, pre):
#     same = 0
#     for i in range(len(test_y)):
#         if test_y[i] == pre[i] or test_y[i] == pre[i] - 1 or test_y[i] == pre[i]+1:
#             same += 1
#     return (same / len(test_y))

# 无+-1
def accuracy(test_y, pre):
    same = 0
    for i in range(len(test_y)):
        if test_y[i] == pre[i]:
            same += 1
    return (same / len(test_y))

In [12]:
accuracy(test_y, pre)

0.7272727272727273